In [ ]:
pip install flask

from flask import Flask, request, jsonify
from pyspark.ml import PipelineModel
from pyspark.sql import SparkSession

app = Flask(__name__)

# Load the PySpark model
spark = SparkSession.builder.appName("StockAnalysis").getOrCreate()
model_path = ""/content/Model.pynb"
model = PipelineModel.load(model_path)

@app.route('/predict', methods=['POST'])
def predict():
    try:
        # Get input data from the request
        input_data = request.json

        # Convert input data to a PySpark DataFrame
        input_df = spark.createDataFrame([input_data])

        # Make predictions using the loaded model
        predictions = model.transform(input_df)

        # Extract the prediction result
        result = predictions.select("prediction").collect()[0]["prediction"]

        return jsonify({"prediction": result})
    except Exception as e:
        return jsonify({"error": str(e)})

if __name__ == '__main__':
    app.run(port=5000)

    
python app.py


import requests

input_data = {
    "DateIndex": 1.0,
    "Open": 300.0,
    "High": 310.0,
    "Low": 295.0,
    "Volume": 1000000.0
}

response = requests.post("http://127.0.0.1:5000/predict", json=input_data)

if response.status_code == 200:
    result = response.json()
    print("Prediction:", result["prediction"])
else:
    print("Error:", response.text)
